# Cái này là để cài đặt folder thành pakege để gọi imoport mà ko lỗi á. Nó tự sinh ra file 'signature_verification.egg-info' chứa thông tin tác giả của mìnhmình

In [1]:
%cd ..

# Cài đặt package
!pip install -e .

# Import module
from utils.helpers import load_config
from losses.triplet_loss import TripletLoss
from models.triplet_network import TripletNetwork

print("Package installed and imported successfully!")

C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
Obtaining file:///C:/Users/HP/PycharmProjects/Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for signature_verification (pyproject.toml): started
  Building editable for signature_verification (pyproject.toml): finished with status 'done'
  Created wheel for signature_verification: filename=signature_verification-0.1.0-0.editable-p


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Package installed and imported successfully!


In [2]:
import os
import torch
import torch.optim as optim
import torchvision.transforms as transforms

In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import random
class TripletSignatureDataset(Dataset):
    def __init__(self, org_dir, forg_dir, transform=None):
        self.org_images = sorted([os.path.join(org_dir, f) for f in os.listdir(org_dir) if f.endswith('.png')])
        self.forg_images = sorted([os.path.join(forg_dir, f) for f in os.listdir(forg_dir) if f.endswith('.png')])
        self.transform = transform
        self.triplets = self._create_triplets()

    def _create_triplets(self):
        triplets = []

        # Tạo triplets với anchor là chữ ký thật
        for anchor in self.org_images:
            # Lấy base name (ví dụ: "10" từ "original_10_1.png")
            base_name = os.path.basename(anchor).split('_')[1]

            # Positive: chữ ký thật khác của cùng người
            positives = [img for img in self.org_images
                        if f"_{base_name}_" in img and img != anchor]

            # Negative: chữ ký giả của cùng người hoặc chữ ký thật của người khác
            forg_negatives = [img for img in self.forg_images if f"_{base_name}_" in img]
            other_negatives = [img for img in self.org_images if f"_{base_name}_" not in img]
            negatives = forg_negatives + other_negatives

            if positives and negatives:
                positive = random.choice(positives)
                negative = random.choice(negatives)
                triplets.append((anchor, positive, negative))

        return triplets

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        anchor_path, positive_path, negative_path = self.triplets[idx]

        anchor = Image.open(anchor_path).convert('L')
        positive = Image.open(positive_path).convert('L')
        negative = Image.open(negative_path).convert('L')

        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return (anchor, positive, negative)

In [4]:
# Transform chung
transform = transforms.Compose([
    transforms.Resize((220, 150)),
    transforms.Grayscale(),  # Đảm bảo ảnh 1 kênh xám
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # 1 kênh -> 3 kênh
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

triplet_dataset = TripletSignatureDataset(
    org_dir=r'C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\signatures\full_org',
    forg_dir=r'C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\signatures\full_forg',
    transform=transform
)

train_size = int(0.8 * len(triplet_dataset))
triplet_train, triplet_test = torch.utils.data.random_split(
    triplet_dataset, [train_size, len(triplet_dataset) - train_size]
)

triplet_train_loader = DataLoader(triplet_train, batch_size=32, shuffle=True)
triplet_test_loader = DataLoader(triplet_test, batch_size=32, shuffle=False)

In [5]:
#Này ae xem sửa thử
config = load_config(r'C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\configs\config_triplet.yaml')
print(config)

{'model': {'backbone': 'resnet18', 'feature_dim': 512}, 'training': {'batch_size': 32, 'learning_rate': 0.001, 'optimizer': 'adam', 'num_epochs': 50, 'margin': 0.5, 'scheduler': 'cosine'}, 'dataset': {'train_data_path': './data/triplet/train/', 'val_data_path': './data/triplet/val/', 'input_size': '(220,150)'}, 'device': 'cuda', 'logging': {'log_dir': './logs/', 'checkpoint_dir': './checkpoints/', 'save_freq': 5}}


In [6]:
#Tiếp theo định nghĩa model từ con config
model = TripletNetwork(config['model']['backbone'], config['model']['feature_dim'])

optimizer = optim.Adam(model.parameters(), lr=config['training']['learning_rate'])
device = torch.device(config['device'] if torch.cuda.is_available() else "cpu")
model.to(device)

C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\PycharmProjects\Deep_Learning-Based_Signature_Forgery_Detection_for_Personal_Identity_Authentication\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TripletNetwork(
  (feature_extractor): ResNetFeatureExtractor(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

AE xem thử rồi cấu trúc ổn không.

In [7]:
#Train kiểu kiểu như vầy ae cứ xem tham thảo

num_epochs = config['training']['num_epochs']
checkpoint_dir = config['logging']['checkpoint_dir']
save_freq = config['logging']['save_freq']

# Tạo thư mục lưu checkpoint nếu chưa có
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for anchor, positive, negative in triplet_train_loader:
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

        anchor_feat, positive_feat, negative_feat = model(anchor, positive, negative)

        loss = TripletLoss()(anchor_feat, positive_feat, negative_feat)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(triplet_train_loader)
    print(f"Epoch [{epoch}/{num_epochs}] - Loss: {avg_loss:.4f}")

    # === SAVE CHECKPOINT ===
    if epoch % save_freq == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")


KeyboardInterrupt: 